# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298411129317                   -0.85    4.1
  2   -8.300214364880       -2.74       -1.25    1.5
  3   -8.300447908575       -3.63       -1.87    3.4
  4   -8.300462406248       -4.84       -2.71    1.0
  5   -8.300464451634       -5.69       -3.22    4.2
  6   -8.300464560196       -6.96       -3.33   10.2
  7   -8.300464597117       -7.43       -3.47    1.0
  8   -8.300464621342       -7.62       -3.61    1.9
  9   -8.300464639078       -7.75       -3.82    1.0
 10   -8.300464641456       -8.62       -3.92    2.6
 11   -8.300464643735       -8.64       -4.23    2.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64192352375                   -0.70    5.5
  2   -16.67869016900       -1.43       -1.14    1.6
  3   -16.67922400298       -3.27       -1.88    2.4
  4   -16.67928138084       -4.24       -2.77    6.5
  5   -16.67928611693       -5.32       -3.14    4.4
  6   -16.67928620878       -7.04       -3.55    1.4
  7   -16.67928622129       -7.90       -4.03    1.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32586655491                   -0.56    6.2
  2   -33.33284924237       -2.16       -1.00    1.1
  3   -33.33585558401       -2.52       -1.68    3.2
  4   -33.33611887783       -3.58       -2.56    3.9
  5   -33.33680963114       -3.16       -2.27    9.5
  6   -33.33682530027       -4.80       -2.30    1.0
  7   -33.33694179916       -3.93       -3.14    6.6
  8   -33.33694283787       -5.98       -3.32    3.5
  9   -33.33694342457       -6.23       -3.49    4.5
 10   -33.33694388896       -6.33       -4.04    2.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298440696099                   -0.85    4.6
  2   -8.300248477415       -2.74       -1.59    1.0
  3   -8.300324675755       -4.12       -2.16    7.4
  4   -8.300329277822       -5.34       -2.19    2.2
  5   -8.300464044187       -3.87       -3.48    2.2
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│      42
│       3
│       3
│       3
│       3
│       3
│       3
│     100
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:36
  6   -8.300464576563       -6.27       -3.89   20.0
  7   -8.300464637770       -7.21       -4.35    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32542976036                   -0.56    5.9
  2   -33.32812016402       -2.57       -1.28    1.0
  3   -25.21909594220   +    0.91       -0.61    7.4
  4   -33.23520973275        0.90       -1.53    9.1
  5   -33.27951823789       -1.35       -1.38    4.4
  6   -33.31060204886       -1.51       -1.80    3.4
  7   -32.83897167880   +   -0.33       -1.22    4.9
  8   -33.33471298544       -0.30       -2.29    4.8
  9   -33.33509962216       -3.41       -2.36    2.2
 10   -33.33605502744       -3.02       -2.40    2.1
 11   -33.33658353153       -3.28       -2.74    1.8
 12   -33.33692764134       -3.46       -3.21    3.0
 13   -33.33693419525       -5.18       -3.36    2.9
 14   -33.33694132361       -5.15       -3.55    2.4
 15   -33.33694226293       -6.03       -3.86    3.2
 16   -33.33694386248       -5.80       -4.28    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.